# 📈 LSTM vs CNN-LSTM Model for Raceway Size Prediction
This notebook demonstrates the comparison between **LSTM** and **CNN-LSTM** models for predicting raceway depth and height using CFD-generated thermal image data.

**Steps:**
1. Load preprocessed data
2. Train LSTM model
3. Train CNN-LSTM model
4. Evaluate and visualize the results
5. Compare the models

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
import matplotlib.pyplot as plt
from math import sqrt

In [ ]:
# Load dataset
df = pd.read_csv('../LSTM.csv')
time_column = 'time'
rates = [100, 120, 140, 150, 160, 180, 200]
features = [f'Depth_up_{r}' for r in rates] + [f'Depth_down_{r}' for r in rates] + [f'Height_{r}' for r in rates]

# Normalize
scaler = MinMaxScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(df[features]), columns=features)

In [ ]:
# Create lag features
def create_lag_features(data, n_lags=5):
    lagged = data.copy()
    for i in range(1, n_lags+1):
        for col in data.columns:
            lagged[f'{col}_lag_{i}'] = data[col].shift(i)
    return lagged.dropna()

n_lags = 5
lagged_df = create_lag_features(scaled_df, n_lags)
lagged_df[time_column] = df[time_column].iloc[n_lags:].values

In [ ]:
train_df = lagged_df[lagged_df[time_column] < 2.8]
test_df = lagged_df[lagged_df[time_column] >= 2.8]

X_train = train_df.drop(features + [time_column], axis=1).values
Y_train = train_df[features].values
X_test = test_df.drop(features + [time_column], axis=1).values
Y_test = test_df[features].values

# Reshape for models
X_train = X_train.reshape((X_train.shape[0], n_lags, X_train.shape[1] // n_lags))
X_test = X_test.reshape((X_test.shape[0], n_lags, X_test.shape[1] // n_lags))

In [ ]:
# Train LSTM model
lstm = Sequential()
lstm.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
lstm.add(Dense(len(features)))
lstm.compile(optimizer='adam', loss='mse')

lstm_history = lstm.fit(X_train, Y_train, epochs=50, verbose=1)
lstm_pred = lstm.predict(X_test)
lstm_pred_inv = scaler.inverse_transform(lstm_pred)
Y_test_inv = scaler.inverse_transform(Y_test)

In [ ]:
# Train CNN-LSTM model
cnn_lstm = Sequential()
cnn_lstm.add(Conv1D(64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
cnn_lstm.add(MaxPooling1D(pool_size=2))
cnn_lstm.add(Flatten())
cnn_lstm.add(Dense(50, activation='relu'))
cnn_lstm.add(Dense(len(features)))
cnn_lstm.compile(optimizer='adam', loss='mse')

cnn_lstm_history = cnn_lstm.fit(X_train, Y_train, epochs=50, verbose=1)
cnn_lstm_pred = cnn_lstm.predict(X_test)
cnn_lstm_pred_inv = scaler.inverse_transform(cnn_lstm_pred)

In [ ]:
# Evaluation metrics
def print_metrics(name, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f'{name} → MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.3f}')

print_metrics('LSTM', Y_test_inv, lstm_pred_inv)
print_metrics('CNN-LSTM', Y_test_inv, cnn_lstm_pred_inv)

In [ ]:
# Plot predictions for one test sample
sample_idx = 0
plt.figure(figsize=(12,5))
plt.plot(Y_test_inv[sample_idx], label='Actual', marker='o')
plt.plot(lstm_pred_inv[sample_idx], label='LSTM', marker='x')
plt.plot(cnn_lstm_pred_inv[sample_idx], label='CNN-LSTM', marker='^')
plt.title('Raceway Size Prediction - Sample 0')
plt.xlabel('Feature Index')
plt.ylabel('Raceway Size (mm)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()